<a href="https://colab.research.google.com/github/nssn96/ML_imageclassifier/blob/main/ML_imageClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Assignment 5 - Image Classification


1.   Author : Surya Narayanan Nadhamuni Suresh
2.   UTA ID : 1001877873


In [ ]:
#import lines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import random
import pickle